In [95]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import pickle
import warnings
import datetime

warnings.filterwarnings('ignore')

In [21]:
links = []
main_link = 'https://www.avito.ru'

In [208]:
def get_links(data, root):
    print('...Function init...')
    for page in range(111,116):
        local_link = '{}/rossiya/mototsikly_i_mototehnika?p={}&view=list'.format(root, page)
        print('...Link {} in process...'.format(local_link))
        soup = BeautifulSoup(requests.get(local_link).text, 'html')
        for item in soup.find_all('a', attrs={'class':'description-title-link'}):
            data.append('{}{}'.format(root, item.get('href')))
        print('...Link parsed...'.format(local_link))
        sleep(random.randint(8, 10))

In [32]:
get_links(links, main_link)

...Function init...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=101&view=list in process...


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=102&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=103&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=104&view=list in process...
...Link parsed...


In [43]:
links = list(set(links))

print(len(links))

for link in links:
    if 'mototsikly_i_mototehnika' in link:
        continue
    else:
        print('False')

10000


In [41]:
with open('links.pickle', 'wb') as f:
    pickle.dump(links, f)

In [188]:
print(links[1366])

soup = BeautifulSoup(requests.get(links[0]).text, 'html')

https://www.avito.ru/moskva/mototsikly_i_mototehnika/skuter_suzuki_skywave_250_1051608917


In [58]:
title = soup.find('span', attrs={'class':'title-info-title-text'}).get_text()
price = soup.find('span', attrs={'class':'js-item-price'}).get_text()

In [72]:
final_price = '{}{}'.format(soup.find('span', attrs={'class':'js-item-price'}).get_text(),
                            soup.find('span', attrs={'class':'font_arial-rub'}).get_text())

In [73]:
final_price

'25 000₽'

In [125]:
date = soup.find('div', attrs={'class':'title-info-metadata-item'}).get_text()

In [123]:
date

'\n \n№ 1307925171, размещено сегодня в 13:08  '

In [128]:
date = 'размещено 24 марта в 13:08'

In [163]:
print(links[3987])

https://www.avito.ru/krasnodar/mototsikly_i_mototehnika/yamaha_drag_star_1100_1503906107


In [80]:
description = soup.find('div', attrs={'class':'item-description-text'}).get_text()

In [81]:
description

'\n  Скутер в хорошем состоянии,стоит новая поршневая,в вариаторе стоит все новое,новая цепь,пластик весь целый но есть не большие царапины,докумены в порядке,торг,по всем вопросам звоните.  '

In [87]:
address = 'Address:\t{}'.format(soup.find('span', attrs={'itemprop':'streetAddress'}).get_text().strip('\n'))

In [88]:
address

'Address:\t Ставропольский край, Изобильненский городской округ, село Московское\n '

In [120]:
month = {'января':'01',
         'февраля':'02',
         'марта':'03',
         'апреля':'04',
         'мая':'05',
         'июня':'06',
         'Июля':'07',
         'августа':'08',
         'сентября':'09',
         'октября':'10',
         'ноября':'11',
         'декабря':'12'}

In [170]:
def get_date(soup):
    try:
        line = (soup.find('div',
                          attrs={'class':'title-info-metadata-item'}).get_text())
        date = line.split('размещено')[1]
        if 'сегодня' in date:
            return datetime.date.today()
        elif 'вчера' in date:
            return datetime.date.today() - datetime.timedelta(1)
        else:
            date = date.split()
            return '2018-{}-{}'.format(month[date[1]], date[0])
    except AttributeError:
        return 'None'
    
def get_title(soup):
    try:
        line = soup.find('span',
                         attrs={'class':'title-info-title-text'}).get_text()
        return line
    except AttributeError:
        return 'None'
    
def get_title(soup):
    try:
        line = soup.find('span',
                         attrs={'class':'title-info-title-text'}).get_text()
        return line
    except AttributeError:
        return 'None'
    

def get_address(soup):
    try:
        address = soup.find('span',
                            attrs={'itemprop':'streetAddress'}).get_text().replace('\n', '')
        return address
    except AttributeError:
        address = 'None'

def get_price(soup):
    try:
        price = '{}{}'.format(soup.find('span',
                                        attrs={'class':'js-item-price'}).get_text(),
                              soup.find('span',
                                        attrs={'class':'font_arial-rub'}).get_text())
        return price
    except AttributeError:
        return 'None'
    
def get_description(soup):
    try:
        description = soup.find('div',
                                attrs={'class':'item-description-text'}).get_text().replace('\n', '')
        return description
    except AttributeError:
        return 'None'

In [167]:
def form_text(link):
    soup = BeautifulSoup(requests.get(link).text, 'html')
    date = get_date(soup)
    address = get_address(soup)
    title = get_title(soup)
    price = get_price(soup)
    description = get_description(soup)
    result = 'Title:\t{}\nDate:\t{}\nAdress:\t{}\nPrice:\t{}\nComment:{}\n'.format(title,
                                                                                     date,
                                                                                     address,
                                                                                     price,
                                                                                     description)
    return result

In [160]:
print(form_text('https://www.avito.ru/livny/mototsikly_i_mototehnika/moped_1472596246'))

Title:	Мопед
Date:	2018-09-28
Adress:	None
Price:	30 000₽
Comment:  Продаются новые кроссоввые мопеды. В наличии осталось 2шт.  



In [162]:
print(form_text('https://www.avito.ru/kaliningrad/mototsikly_i_mototehnika/daelim_roadwin_125_2008_g._v_465758898'))

Title:	Daelim roadwin 125 2008 г. в
Date:	2018-09-27
Adress:	 Калининград, район Ленинградский, улица Александра Невского, 120Б 
Price:	110 000₽
Comment:  Продам мотоцикл DAELIM  ROADWIN 125 2008 г.в.  в очень хорошем состоянии и небольшим пробегом, в оригинальном раскрасе, весь пластик целый. Растаможен на область, можно при желании дотаможить на РФ плюс 26т.р. к цене. торг при осмотре. Фирма БАЙК СИТИ принимает на зимнее хранение, 1500 р. месяц. А так же заказы с сайта ALEGRO.  В наличии до пятидесяти единиц мототехники. Фирма БАЙК СИТИ приглашает.А так же работаем под заказ.  



In [168]:
print(form_text('https://www.avito.ru/krasnodar/mototsikly_i_mototehnika/yamaha_drag_star_1100_1503906107'))

Title:	Yamaha Drag Star 1100
Date:	2018-09-28
Adress:	 микрорайон Центральный, Краснодар, район Центральный, улица Дмитриевская Дамба 
Price:	265 000₽
Comment:
  Продам Yamaha Drag Star 11002001годПробег 39646Техническое состояние отличное, сел и поехал, падений не было. Хром весь целый, сидения целые. Есть потёртости на баке от наколенников, вопрос решаемый. Новый акум. Кофры. Есть спинка, но нужна перетяжка. Торг реальному покупателю.  



In [195]:
for link in links[8539:]:
    file_name = link.split('/')[len(link.split('/')) - 1]
    with open('avito_files/' + str(file_name) + '.txt', 'w') as f:
        try:
            f.write(form_text(link))
            print('Link {} parsed. Saved as "{}"'.format(num, str(file_name)))
        except IndexError:
            print('>>> BROKEN LINK N.{} <<<'.format(num))
        num += 1
        sleep(random.randint(8, 10))

Link 8539 parsed. Saved as "yamaha_jog_1180718882"
Link 8540 parsed. Saved as "motoroller_elektron_raritet_sovetskogo_proizvo_1673985859"
Link 8541 parsed. Saved as "mototsikl_b8700_bmw_r1150r_rockstar_iz_yaponii_1657302516"
Link 8542 parsed. Saved as "prodam_pit-bayk_racer_125_pe_torg_1679320556"
Link 8543 parsed. Saved as "giperbayk_1259766082"
Link 8544 parsed. Saved as "virus_x2_1085776656"
Link 8545 parsed. Saved as "motovezdehod_polaris_ranger_570_1297208697"
Link 8546 parsed. Saved as "jog_c_1000492198"
Link 8547 parsed. Saved as "snegohod_brp_ski-doo_gtx_550_1348770080"
Link 8548 parsed. Saved as "aro_10_628310071"
Link 8549 parsed. Saved as "stels_500_gt_2014_1036781049"
Link 8550 parsed. Saved as "yamaha_rx-1_mtx_1316330644"
Link 8551 parsed. Saved as "tundra_lt_600_ace_1509816757"
Link 8552 parsed. Saved as "suzuki_sv650_929340338"
Link 8553 parsed. Saved as "racer_rc250cs_1287533751"
Link 8554 parsed. Saved as "mototsikl_bmw_f650st_1998_1483702942"
Link 8555 parsed. Saved a

Link 8675 parsed. Saved as "ural_m-67-36_1347705278"
Link 8676 parsed. Saved as "prodam_yamahu_1398254967"
Link 8677 parsed. Saved as "prodam_mototsikl_honda_xrl250_baja_1994g_1275908087"
Link 8678 parsed. Saved as "bmw_r_nine_t_1393058122"
Link 8679 parsed. Saved as "yamaha_phazer_m-tx_1364792885"
Link 8680 parsed. Saved as "harley-davidson_street_glide_1249444959"
Link 8681 parsed. Saved as "elektro-samokat_1471699615"
Link 8682 parsed. Saved as "mototsikl_sssr_izh_yupiter_2k_1387627678"
Link 8683 parsed. Saved as "voshod_2m_937314484"
Link 8684 parsed. Saved as "prodam_alfu_1526323005"
Link 8685 parsed. Saved as "kvadr_150_kub_1265330001"
Link 8686 parsed. Saved as "skuter_tachila_982840625"
Link 8687 parsed. Saved as "kawasaki_ninja_zx-10r_2006_1592066812"
Link 8688 parsed. Saved as "irbis_ttr250_1408985612"
Link 8689 parsed. Saved as "honda_cbr_600_f4i_1258373326"
Link 8690 parsed. Saved as "honda_cbr600rr_2007_1464403896"
Link 8691 parsed. Saved as "polaris_pro-rmk_800_155_le_151

Link 8808 parsed. Saved as "voshod_3m_1309991265"
Link 8809 parsed. Saved as "ural_m67_1683435625"
Link 8810 parsed. Saved as "pitbayk_apollo_150_1332930462"
Link 8811 parsed. Saved as "yamaha_grizzly_700_1487825363"
Link 8812 parsed. Saved as "tayga_varyag_550v_1278185393"
Link 8813 parsed. Saved as "novyy_snegohod_buran_ade_lider_663240263"
Link 8814 parsed. Saved as "tinger_ural_1181891815"
Link 8815 parsed. Saved as "50ss_s_dokumentami_1351057808"
Link 8816 parsed. Saved as "skay_vey_suzuki_2006_god_801599772"
Link 8817 parsed. Saved as "aprilia_rs_125_2009g_934358579"
Link 8818 parsed. Saved as "suzuki_bandit_1365287123"
Link 8819 parsed. Saved as "cbr600_obmen_na_endurokross_1508954561"
Link 8820 parsed. Saved as "zapchasti_kavasaki_zzr_1100_1585662730"
Link 8821 parsed. Saved as "mototsikl_yamaha_warrior_1683373969"
Link 8822 parsed. Saved as "aprilia_rsv_1000_1607837746"
Link 8823 parsed. Saved as "honda_cb1300sf_2003_g.v_1713910669"
Link 8824 parsed. Saved as "vezdehod_1524572

Link 8942 parsed. Saved as "giroskuter_6.5_1033758651"
Link 8943 parsed. Saved as "kvadrotsikl_1109875228"
Link 8944 parsed. Saved as "gusenichnyy_vezdehod_992551667"
Link 8945 parsed. Saved as "zonda_takt_af_30_1692804681"
Link 8946 parsed. Saved as "snegohod_brp_ski-doo_freeride_800_e-tec_1057625090"
Link 8947 parsed. Saved as "izh_yupiter_5_1465738891"
Link 8948 parsed. Saved as "prodaetsya_moped_recer_rc110n_1598161181"
Link 8949 parsed. Saved as "mototsikl_1445436793"
Link 8950 parsed. Saved as "cf_moto_650nk_1178529396"
Link 8951 parsed. Saved as "honda_2003g._cb_400_sf_2_vtek_1528826242"
Link 8952 parsed. Saved as "snegohod_1199704837"
Link 8953 parsed. Saved as "omaks_200_1359720835"
Link 8954 parsed. Saved as "prodam_motoroller_muravey_1565437478"
Link 8955 parsed. Saved as "honda_cb500_1718831561"
Link 8956 parsed. Saved as "prodam_snegohod_1185829896"
Link 8957 parsed. Saved as "bmw_f650gs_1542834413"
Link 8958 parsed. Saved as "prodam_minsk_12v_1235861973"
Link 8959 parsed.

Link 9076 parsed. Saved as "prodam_zapchast_1445785182"
Link 9077 parsed. Saved as "prodam_rys_1545457612"
Link 9078 parsed. Saved as "kayo_125_1035423927"
Link 9079 parsed. Saved as "gusenitsa_i_lyzhy_na_yamahu_1620658829"
Link 9080 parsed. Saved as "suzuki_boulevard_c90_1120128688"
Link 9081 parsed. Saved as "honda_cbr_600_f4i_2002_949679579"
Link 9082 parsed. Saved as "izh_planeta_s_kolyaskoy_1125659771"
Link 9083 parsed. Saved as "honda_trx680_1475787301"
Link 9084 parsed. Saved as "prodazhasuzuki_burgman_skywave_250_1622010591"
Link 9085 parsed. Saved as "xmoto_raptor_200_zw250-1_1161470177"
Link 9086 parsed. Saved as "prodam_snegohod_polaris_widetrak_xl_1688044963"
Link 9087 parsed. Saved as "kayo_140_classic_1439606452"
Link 9088 parsed. Saved as "prodam_mototsikl_dnepr_11_1176229713"
Link 9089 parsed. Saved as "motolend_mh250_1305037338"
Link 9090 parsed. Saved as "prodam_skuter_honda_giorno_1098695446"
Link 9091 parsed. Saved as "prodam_dnepr_1494151872"
Link 9092 parsed. Save

...

Link 9340 parsed. Saved as "elektricheskiy_seev_sitycoco_x7br_12ah_1000w_926294257"
Link 9341 parsed. Saved as "izh_planeta_4_965099957"
Link 9342 parsed. Saved as "yamaha_tf_872436527"
Link 9343 parsed. Saved as "riga_16_958987534"
Link 9344 parsed. Saved as "mototsikl_honda_cb_1_1095856820"
Link 9345 parsed. Saved as "lynx_rave_re_800_e-tec_961083324"
Link 9346 parsed. Saved as "prodayu_snegobolotohod_pelets_kruizer_1083949344"
Link 9347 parsed. Saved as "prodam_mototsikl_honda_vtr_1000_1407167795"
Link 9348 parsed. Saved as "skutor_honda_dio_80_kubov_1693685871"
Link 9349 parsed. Saved as "briar_streetfire_1075619156"
Link 9350 parsed. Saved as "prodayu_mototsikl_1258125084"
Link 9351 parsed. Saved as "mototsikl_voshod_1_pervyh_seriy_vypuska_1003177868"
Link 9352 parsed. Saved as "bmw_hp4_s1000rr_2013_1012288333"
Link 9353 parsed. Saved as "stels_flex_250_1419822446"
Link 9354 parsed. Saved as "mototsikl_krossovyy_avantis_enduro_250_pro_2118_12889329"
Link 9355 parsed. Saved as "ste

Link 9472 parsed. Saved as "kawasaki_zx636r_2005_1439935692"
Link 9473 parsed. Saved as "stels_leopard_600_otval_1578345943"
Link 9474 parsed. Saved as "yamaha_1695972091"
Link 9475 parsed. Saved as "kvadrotsikl_brp_maverick_trail_800_dps_1428055831"
Link 9476 parsed. Saved as "skuter_950486036"
Link 9477 parsed. Saved as "tritsikl_3h3_1211619393"
Link 9478 parsed. Saved as "motoland_alpha_110cc_1413870787"
Link 9479 parsed. Saved as "honda_dio_af_35_zx_iz_yaponiya_1568827681"
Link 9480 parsed. Saved as "racer_rc-300_gy-8_1686635020"
Link 9481 parsed. Saved as "suzuki_bandit_400_1310487656"
Link 9482 parsed. Saved as "honda_dio_34_960615578"
Link 9483 parsed. Saved as "snegohod_yamaha_rs_venture_tf_1096181968"
Link 9484 parsed. Saved as "snegohod_rys_500mp_1638523543"
Link 9485 parsed. Saved as "m-103_973870543"
Link 9486 parsed. Saved as "riga_13_1074234097"
Link 9487 parsed. Saved as "kawasaki_klx_250_1519070195"
Link 9488 parsed. Saved as "husqvarna_wre125_1637089734"
Link 9489 pars

Link 9609 parsed. Saved as "kvadrotsikl_yacota_kids_8_862054399"
Link 9610 parsed. Saved as "cobra_crossfire_125_1287921309"
Link 9611 parsed. Saved as "skuter_suzuki_leats_japan_971467758"
Link 9612 parsed. Saved as "skuter_1475074726"
Link 9613 parsed. Saved as "honda_tact_af30_yaponskiy_skuter_789995884"
Link 9614 parsed. Saved as "arctic_cat_prowler_700hdx_1234443023"
Link 9615 parsed. Saved as "prodam_mototsikl_1427417010"
Link 9616 parsed. Saved as "shtorm_1315376815"
Link 9617 parsed. Saved as "prodayu_izh_yupiter_-3_843062135"
Link 9618 parsed. Saved as "cbr_919_rr_1074926122"
Link 9619 parsed. Saved as "harley-davidson_sportster_1200s_1037707895"
Link 9620 parsed. Saved as "ural_strit-bayk_1505323286"
Link 9621 parsed. Saved as "honda_magna_750_1423544821"
Link 9622 parsed. Saved as "prodam_mototsikl_bez_dokumentov_1347075397"
Link 9623 parsed. Saved as "sym_volf_22014g.probeg_22t_250_kubov_obmena_net_1695797311"
Link 9624 parsed. Saved as "woqu_elektricheskiy_skuter_mototsikl

Link 9744 parsed. Saved as "kvadrotsikl_zim-350_1407612656"
Link 9745 parsed. Saved as "prodam_ural_12v_1118351637"
Link 9746 parsed. Saved as "honda_lead_100_1219111181"
Link 9747 parsed. Saved as "prodam_honda_dio_1002813259"
Link 9748 parsed. Saved as "mototsikl_1008485060"
Link 9749 parsed. Saved as "giroskutor_1719515219"
Link 9750 parsed. Saved as "alfa_2_sht_probeg_500_km_1592727076"
Link 9751 parsed. Saved as "arctic_cat_240ls_998043223"
Link 9752 parsed. Saved as "kavasaki_zx_10_r_1283875090"
Link 9753 parsed. Saved as "suzuki_sv_400_2007_g._v_642356115"
Link 9754 parsed. Saved as "suzuki_gemma_1563149953"
Link 9755 parsed. Saved as "omaks_125-26_1298926653"
Link 9756 parsed. Saved as "skuter_honda_zoomer_af58_1329957999"
Link 9757 parsed. Saved as "kvadrotsikl_russkaya_mehanika_rm500_1712943341"
Link 9758 parsed. Saved as "honda_shadow_vt_750_626272955"
Link 9759 parsed. Saved as "atv_200_1237029918"
Link 9760 parsed. Saved as "bmw_k100rt_1154812747"
Link 9761 parsed. Saved a

Link 9879 parsed. Saved as "prodam_ili_obmenyayu_izh_yupiter_5_1723670963"
Link 9880 parsed. Saved as "bmw_f650cs_1529627515"
Link 9881 parsed. Saved as "izh_yupiter_5_1567970534"
Link 9882 parsed. Saved as "moped_karpaty_1583294113"
Link 9883 parsed. Saved as "yamaha_xv_400_virago_1226602990"
Link 9884 parsed. Saved as "kavasaki_250_1670692854"
Link 9885 parsed. Saved as "kawasaki_zx6-r_1307987976"
Link 9886 parsed. Saved as "prodam_skuter_suzuki_skyweve_250_1658579224"
Link 9887 parsed. Saved as "moped_alfa_1344450394"
Link 9888 parsed. Saved as "snego_boloto_hod_pioner_1657259330"
Link 9889 parsed. Saved as "prodam_skuter_1565728181"
Link 9890 parsed. Saved as "snegohod_arctic_cat_m800_limited_1351381917"
Link 9891 parsed. Saved as "prodam_kvadrotsikl_cfmoto_x5_h.o._eps_1322518081"
Link 9892 parsed. Saved as "prodayu_honda_dio_34_1178630999"
Link 9893 parsed. Saved as "mini_pitbayk_crf50_1569064316"
Link 9894 parsed. Saved as "mototsikl_1437053450"
Link 9895 parsed. Saved as "motots

In [209]:
new_links = []
new_links2 = []

In [202]:
get_links(new_links, main_link)

...Function init...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=107&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=108&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=109&view=list in process...
...Link parsed...


In [204]:
len(new_links)

300

In [205]:
new_links = [link for link in new_links if link not in links]

In [206]:
len(new_links)

269

In [216]:
for link in new_links2:
    file_name = link.split('/')[len(link.split('/')) - 1]
    with open('avito_files/' + str(file_name) + '.txt', 'w') as f:
        try:
            f.write(form_text(link))
            print('Link {} parsed. Saved as "{}"'.format(num, str(file_name)))
        except IndexError:
            print('>>> BROKEN LINK N.{} <<<'.format(num))
        num += 1
        sleep(random.randint(8, 10))

Link 10269 parsed. Saved as "prodayu_kvadrotsikl_stels_gt_500_1139123478"
Link 10270 parsed. Saved as "skuter_honda_gyro_canopy_trehkolesnyy_1190135587"
Link 10271 parsed. Saved as "maksiskuter_suzuki_skywave_250_-_s3_2008_bp_po_rf_1317167121"
Link 10272 parsed. Saved as "mini_choper_1651199497"
Link 10273 parsed. Saved as "snegohod_1238259359"
Link 10274 parsed. Saved as "honda_cb400_1059047284"
Link 10275 parsed. Saved as "honda_cbn_600_1365102629"
Link 10276 parsed. Saved as "mototsikl_ural_imz-8.103_1269003136"
Link 10277 parsed. Saved as "skuter_1585718920"
Link 10278 parsed. Saved as "kayo_t6_1265375762"
Link 10279 parsed. Saved as "krutoy_mototsikl_1217329223"
Link 10280 parsed. Saved as "moto_na_lyubitelya_1546807765"
Link 10281 parsed. Saved as "mototsikl_honda_cb600_hornet_1178741752"
Link 10282 parsed. Saved as "yamaha_tdm850_bez_probega_po_rf_1636859165"
Link 10283 parsed. Saved as "stels_gepard_800_1722753475"
Link 10284 parsed. Saved as "honda_vfr750_1288936537"
Link 1028

Link 10401 parsed. Saved as "arctic_cat_7000xt_1298631438"
Link 10402 parsed. Saved as "suzuki_bandit_gsf400_1646448243"
Link 10403 parsed. Saved as "yamaha_xv1700rs_1399588852"
Link 10404 parsed. Saved as "mini_snegohod_vyuga_m_1704199532"
Link 10405 parsed. Saved as "cf_moto_x6_1524267717"
Link 10406 parsed. Saved as "ural_1594999959"
Link 10407 parsed. Saved as "honda_lead_1186176879"
Link 10408 parsed. Saved as "suzuki_boulevard_c50t_2012g_1209647130"
Link 10409 parsed. Saved as "moped_delta_1319694387"
Link 10410 parsed. Saved as "orion_1723916119"
Link 10411 parsed. Saved as "izh_yupiter_5_1159198237"
Link 10412 parsed. Saved as "mototsikl_baltmotors_motard_200_595001271"
Link 10413 parsed. Saved as "pitbayki_motoland_crf125._novye_garantiya_1642561729"
Link 10414 parsed. Saved as "mototsiklmoped_alfa_rubikon_2018_49110_disk_torm_1580408524"
Link 10415 parsed. Saved as "skuter_honling_priboy_50_1234639577"
Link 10416 parsed. Saved as "vsedorozhnyy_mototsikl_comandor_200_s_pts_594

KeyboardInterrupt: 

In [210]:
get_links(new_links2, main_link)

...Function init...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=111&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=112&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=113&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=114&view=list in process...
...Link parsed...
...Link https://www.avito.ru/rossiya/mototsikly_i_mototehnika?p=115&view=list in process...
...Link parsed...


In [213]:
for link in new_links:
    links.append(link)

In [214]:
new_links2 = [link for link in new_links2 if link not in links]

In [215]:
len(new_links2)

475

In [217]:
for link in new_links2:
    links.append(link)

In [218]:
len(links)

10475

In [222]:
links = list(set(links))
n1 = 1

In [225]:
for link in links[1:]:
    file_name = link.split('/')[len(link.split('/')) - 1]
    try:
        with open('avito_files/' + str(file_name) + '.txt', 'a') as f:
            f.write('Link:\t{}\n'.format(str(link)))
            print('File {} updated'.format(str(file_name)))
            n1 += 1
    except Exception:
        continue

File stels_sb200_1247436004 updated
File skuter_dlya_prodazhi_1504991932 updated
File kvadrotsikl_irbis_atv150u_1463856208 updated
File support_yamaha_rhino_921796670 updated
File kvadrotsikl_1320334652 updated
File honda_cb_400cb1_1713512092 updated
File yamaha_viking_540_4_2014god_1695170659 updated
File racer_rc50_alfa_1524864052 updated
File mototsikl_1555736821 updated
File tayga_ataka_551_ii_2018_1220282443 updated
File yamaha_wr_450_f_1513116252 updated
File honda_cbr_600rr_1641152561 updated
File moped_alfa_1342634200 updated
File honda_f2_cbr_600_1512828062 updated
File honda_gold_wing_gl_1800_1291497816 updated
File prodam_gotovuyu_ramu_deltaplana_s_vodyanymi_lyzhami_1060836093 updated
File cfmoto_x8_h.o._eps_1079802209 updated
File pitbayk_fighter_125_1409479714 updated
File dnepr_936745464 updated
File honda_fit_1685473393 updated
File karting_1296517710 updated
File alpha_1217837790 updated
File prodam_skuter_1176646761 updated
File vezdehod_na_baze_uaz_trekol-39041_117759

File honda_cbr_1000_ra_930882078 updated
File kayo_140_1675615099 updated
File skuter_suzuki_b._u_1403252698 updated
File skuter_1531790720 updated
File prodam_snegohod_yamaha_vk_professional_2_1494718822 updated
File honda_cbr_600_f4i_1555388846 updated
File arti_cat_1480573421 updated
File sym_jet_4_125_991544165 updated
File mototsikl_ural_1703579075 updated
File vezdehod_776942887 updated
File prodayu_kvadrotsikl_motovezdehod_1560528460 updated
File motosobaka_1017704118 updated
File delta_dirt_bike_orion_apollo_2018_1494106527 updated
File karpaty_s_motorom_ot_minskogo_1494689557 updated
File tsf_moto_800_1536328348 updated
File kawasaki_zzr600_zzr_600_1405516088 updated
File izh_planeta_1055220366 updated
File samohodnoe_samosobrannoe_shassi_865931996 updated
File vezdehod_1349479311 updated
File stels_outlander_150_1073097831 updated
File elektrosamokat_1606531806 updated
File izh_yupiter_2_1064886232 updated
File prodam_yamaha_fz_400_1478042744 updated
File polaris_dragon_700_2

File prodam_moped_1300749643 updated
File prodam_snegohod_brp_lynx_adventure_gt_900_1124489710 updated
File voshod_1321288233 updated
File honda_cbr1000f_1222194082 updated
File prodam_yamaha_fzx_750_1311913087 updated
File prodam_honda_drozd_1353005041 updated
File prodam_honda_dzhorno_af_77_honda_giorno_af77_1703224156 updated
File kawasaki_zx636r_1329710104 updated
File yamaha_xvs_1300_2013g_1569980445 updated
File irbis_atv_250s_1521839355 updated
File prodaetsya_harley_davidson_custom_1356085645 updated
File yava_350_1228273460 updated
File prodayu_moped_reyser_1306537076 updated
File detskie_benzinovye_i_elektricheskie_kvadrotsikly_atv_1603160585 updated
File orion_125_1066359886 updated
File yupiter_5_1043760773 updated
File yamaha_srx_700_1163328167 updated
File honda_cbr600_f3_iz_germanii._bez_probega_po_ry_1655270099 updated
File mototsikl_bajaj_avenger_220_street_2018_1588735467 updated
File honda_cbf_600_2008g_inzhektor_1706961504 updated
File piaggio_mp3_250_kub._2009_g_11

File rm_vector_551i_1286302938 updated
File prodam_mototsikl_suzuki_intruder_800_1562768810 updated
File yamaha_royal_star_1300_949046978 updated
File cf_moto_x6_crazy_1585073260 updated
File motobuksirovschik_muhtar_15l.s_novinka_2018_1638118560 updated
File motobuksirovschik_-_motosobaka_s_variatorom_15_l.s_336937604 updated
File karbyuratory_honda_cb-400_sf_version_s_1714139286 updated
File honda_vfr800_obmen_na_avto_1457628436 updated
File prodam_skuter_984987645 updated
File baggi_yacota_ranger_200_1099181434 updated
File ducati_multistrada_1100_s_turist_na_kazhdyy_den_955937377 updated
File zid_pilot_3-h_kolesnyy_1335989632 updated
File suzuki_gsx-r_750_2007_2006_k6_k7_1306495036 updated
File moped_honda_dio_2007_1312549798 updated
File novyy_kvadrotsikl_linhai_yamaha_d_300_4x4_1017869878 updated
File elektrosamokat_1295597556 updated
File suzuki_rf_400rv_1499986230 updated
File suzuki_intruder_1400_1396611406 updated
File prodam_skuter_1316115543 updated
File yupiter_5_151879800

File prodam_kvadrotsikl_yamaha_grizli_yfm_550_1652512725 updated
File motobuksirovschik_motosobaka_487667887 updated
File mototsikl_bajaj_boxer_bm_150_x_1049584214 updated
File ducati_monster_934589968 updated
File yamaha_wr400f_1289265722 updated
File zapchasti_i_remont_skuterov_i_mopedov_252688 updated
File sportster_1363693137 updated
File honda_cbr250r_1462667715 updated
File prodayu_moped_110kubov_1238549138 updated
File moped_stingray_1602207815 updated
File yamaha_xvs_650_a_1126789217 updated
File mototsikl_no._b7468_honda_vt1300_cx_iz_yaponii_529741535 updated
File honda_magna_vf750v45_1394501860 updated
File honda_lead_100cc_1298202965 updated
File prodam_brp-maverick_h_2013g_1535169837 updated
File elektroskuter_1545457925 updated
File falkon_desperado_250_i_svk_sport_150_1323312274 updated
File suzuki_djebel_250_990793552 updated
File bolotohod_1592681428 updated
File brp_outlander_800_1289518043 updated
File kvadrotsikl_armada_atv_150_b_1296782894 updated
File skuter_tachil

...

File skutermoped_honda_dio_af34_c_konteynera_1426130958 updated
File prodam_racer_gs_110_1180762858 updated
File avantis_250_1498004809 updated
File xonda_xr_250_1588882365 updated
File bmw_f_650_st1997_g._bez_probega_po_rf_1336149192 updated
File honda_cb_250f_jade_1257467739 updated
File atlant_150_1213628113 updated
File rm_800_1075325955 updated
File drift_trayk_drift_trike_1009746660 updated
File elektroskuter_smart_setr_wh-e11_1691312773 updated
File suzuki_skywave_400_2001_god_1256865466 updated
File indigo_shtorm_125_1469824935 updated
File honda_nv400cp_1290789956 updated
File snegohod_tayga_klassik_1319953384 updated
File prokat_kvadrotsiklov_snegohodov_1333046129 updated
File honda_cb_750-2_935375413 updated
File skuter_shtorm_korichnevyy_734112626 updated
File alfa_87111469 updated
File honda_vtr_1000_sp-2_rc51_1578181642 updated
File rm_800_utv_s_probegom_1548261695 updated
File kayo_140_1265244708 updated
File buran_ae_ade_lider_669193384 updated
File kayo_t6_1265375762 u

File mototsikl_krossovyy_avantis_enduro_250_pro_2118_12889329 updated
File stels_500k_1541230114 updated
File motovelosiped_1492455040 updated
File skuter_speedmax_50cc_1158518080 updated
File mototsikl_suzuki_gsf_1200s_bandit_1305112618 updated
File maped_1381259918 updated
File skuter_50_kubovyy_1337678771 updated
File yamaha_gir_1452776079 updated
File dnepr_mt_11_1987g_1196472058 updated
File snegohod_tayga_varyag_500_horoshee_sostoyanie_1092984048 updated
File zadniy_kofr_dlya_kvadrotsikla_ogio_novyy_933309131 updated
File karakat_1386127860 updated
File x-moto_raptor_250_1047971263 updated
File skuter_eagle_1012132873 updated
File prodazha_ili_obmen_izh_planeta_3_1166291904 updated
File minsk_1610091764 updated
File kawasaki_er-6n201213200_probeg_1160254631 updated
File mototsikl_wels_ghost_250cc_1132155604 updated
File honda_cbr600rr_v_idealnom_sostoyanii_1379955269 updated
File pvz_16vm_motovelosiped_1493805629 updated
File honda_xl1000_varadero_2005g_793149697 updated
File pro

File suzuki_gsf_1200_n_bandit_1075991556 updated
File skuter_50_kubov_1660220743 updated
File brp_summit_163_800_prodam_1389852022 updated
File kvadrotsikl_polyaris_500_n.o._2012g.e_1709346670 updated
File honda_cbr_929_rr_1048375565 updated
File snegohod_tayga_ataka_551_844064594 updated
File sani_dlya_rybalki_1005950421 updated
File moped_yamasaki_2012_g._vyp_850821840 updated
File prodaetsya_vezdehod_analog_argo_vild_panter_1242766675 updated
File abm_rx200_sm_18ls_1211156913 updated
File prodam_mototsikl_izh_planeta_5_1468591959 updated
File kawasaki_er-6f_1214180766 updated
File kayo_140_1631483736 updated
File stels_arrow_998926574 updated
File brp_ski_do_600_adrenalin_115_sil_1673340016 updated
File kawasaki_zx600e_1352263862 updated
File mototsikl_voshod_3m_1564917880 updated
File snegohod_polaris_800_switchback_assault_144_1337290107 updated
File krossovyy_mototsikl_931780873 updated
File mototsikl_wrx_250_pit_1460106875 updated
File honda_cbr_1000_rr_1287082708 updated
File p

File honda_lead_jf06_100_1548681277 updated
File stels_vortex_150_1676055454 updated
File prodam_izh_yupiter3_nuzhna_zdelat_zaryadku_1022398305 updated
File muravey_1724804394 updated
File motovezdehod_polaris_850_touring_1189261618 updated
File ktm_525_sx_972666229 updated
File prodam_kolyasku_na_dnepr_1637577071 updated
File motobuksirovschik_muzhik-760_vsesezonnyy_1538785593 updated
File kvadrotsikl_zim-350_1407612656 updated
File prodam_ural_12v_1118351637 updated
File honda_lead_100_1219111181 updated
File prodam_honda_dio_1002813259 updated
File mototsikl_1008485060 updated
File giroskutor_1719515219 updated
File alfa_2_sht_probeg_500_km_1592727076 updated
File arctic_cat_240ls_998043223 updated
File kavasaki_zx_10_r_1283875090 updated
File suzuki_sv_400_2007_g._v_642356115 updated
File suzuki_gemma_1563149953 updated
File omaks_125-26_1298926653 updated
File skuter_honda_zoomer_af58_1329957999 updated
File kvadrotsikl_russkaya_mehanika_rm500_1712943341 updated
File honda_shadow_

File skuter_honda_dio_af27_tolko_iz_yaponii_867131296 updated
File honda_dio_skuter_1544028121 updated
File brp_can-am_maverick_x3_x_mr_turbo_r_2018_1025478699 updated
File honda_crf450r_2017g_983694054 updated
File skutery_1326915318 updated
File mototsikl_kawasaki_ninja_650_2018_1169056159 updated
File gx-r_250_1128390374 updated
File honda_af-34_1557712104 updated
File mototsikl_1119299138 updated
File arctic_cat_m8_2009g_trak_162_729201751 updated
File victory_citi_jet_100cc_1703151640 updated
File detskiy_kvadrik_s_elektrostarterom_motax_x-16_1074617917 updated
File stels_700_dinli_1320605243 updated
File honda_cbr600f4_1336040472 updated
File prodam_mototsikl_yava-350_1269865260 updated
File kawasaki_zzr_600_1006234897 updated
File aprilia_pegaso_650_1707934541 updated
File izh_yupiter_5_1312750439 updated
File yamaha_cygnus_125_x_1416399673 updated
File yamaha_r1_1414837075 updated
File tornado_49kubov_1463774019 updated
File rm_500-2_1600465936 updated
File ktm_sx85_2016_876031